In [3]:
import ROOT
from ROOT import TMVA


In [4]:
modelFile = "Higgs_trained_model.h5"
if (ROOT.gSystem.AccessPathName(modelFile)) :
    ROOT.Info("TMVA_SOFIE_RDataFrame","You need to run TMVA_Higgs_Classification.C to generate the Keras trained model")
    

In [5]:
# parse the input Keras model into RModel object
model = ROOT.TMVA.Experimental.SOFIE.PyKeras.Parse(modelFile)

Keras Version: 2.8.0
Model has not a defined batch size, assume is 1 - input shape : { 1 , 7 }


2022-04-09 15:05:47.726523: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/neel/Root/install/lib:/opt/ros/noetic/lib
2022-04-09 15:05:47.726548: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-09 15:05:51.402104: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/neel/Root/install/lib:/opt/ros/noetic/lib
2022-04-09 15:05:51.402135: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-09 15:05:51.402159: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on t

In [6]:
#Generating inference code
model.Generate()
model.OutputGenerated()
model.PrintGenerated()

//Code generated automatically by TMVA for Inference of Model file [Higgs_trained_model.h5] at [Sat Apr  9 09:35:47 2022] 
#include<algorithm>
#include<vector>
#include<cmath>
#include "TMVA/SOFIE_common.hxx"
#include <fstream>

namespace TMVA_SOFIE_Higgs_trained_model{
namespace BLAS{
	extern "C" void sgemv_(const char * trans, const int * m, const int * n, const float * alpha, const float * A,
	                       const int * lda, const float * X, const int * incx, const float * beta, const float * Y, const int * incy);
	extern "C" void sgemm_(const char * transa, const char * transb, const int * m, const int * n, const int * k,
	                       const float * alpha, const float * A, const int * lda, const float * B, const int * ldb,
	                       const float * beta, float * C, const int * ldc);
}//BLAS
struct Session {
std::vector<float> fTensor_dense4bias0 = std::vector<float>(2);
float * tensor_dense4bias0 = fTensor_dense4bias0.data();
std::vector<float> fTensor

In [7]:
# now compile using ROOT JIT trained model
print("compiling SOFIE model ")
generatedFile = "Higgs_trained_model.hxx"
modelName = "Higgs_trained_model"
ROOT.gInterpreter.Declare('#include "' + generatedFile + '"')



compiling SOFIE model 


True

In [8]:
ROOT.TMVA.Tools.Instance()
TMVA.PyMethodBase.PyInitialize()

outputFile = ROOT.TFile.Open("Higgs_data_Output.root", "RECREATE")

In [9]:
inputFileName = "Higgs_data.root"
inputFile = ROOT.TFile.Open(inputFileName)
inputFileLink = "http://root.cern.ch/files/" + inputFileName

if (ROOT.gSystem.AccessPathName(inputFileName)!=None):
    # file exists
    inputFile = ROOT.TFile.Open( inputFileName )
if(inputFile == None): 
# download file from Cernbox location
    ROOT.Info("TMVA_Higgs_Classification","Download Higgs_data.root file")
    ROOT.TFile.SetCacheFileDir(".")
    inputFile = ROOT.TFile.Open(inputFileLink, "CACHEREAD")
    if (inputFile == NULL):
        Error("TMVA_Higgs_Classification","Input file cannot be downloaded - exit")

In [10]:
signalTree     = inputFile.Get("sig_tree")
backgroundTree = inputFile.Get("bkg_tree")

signalTree.Print()
# Get branch "var1" as numpy array
# data = signalTree.AsNumpy(["0"])

******************************************************************************
*Tree    :sig_tree  : tree                                                   *
*Entries :    10000 : Total =         1177229 bytes  File  Size =     785298 *
*        :          : Tree compression factor =   1.48                       *
******************************************************************************
*Br    0 :Type      : Type/F                                                 *
*Entries :    10000 : Total  Size=      40556 bytes  File Size  =        307 *
*Baskets :        1 : Basket Size=    1500672 bytes  Compression= 130.54     *
*............................................................................*
*Br    1 :lepton_pT : lepton_pT/F                                            *
*Entries :    10000 : Total  Size=      40581 bytes  File Size  =      30464 *
*Baskets :        1 : Basket Size=    1500672 bytes  Compression=   1.32     *
*...................................................

In [11]:
df1 = ROOT.RDataFrame("sig_tree", "Higgs_data.root")
df2 = ROOT.RDataFrame("bkg_tree", "Higgs_data.root")

In [12]:
# Take all columns
npy1 = df1.AsNumpy()
print(npy1)
print("--------------------------------------")
npy2 = df2.AsNumpy()
print(npy2)

{'Type': ndarray([1., 1., 1., ..., 1., 1., 1.], dtype=float32), 'jet1_b-tag': ndarray([0.       , 2.1730762, 0.       , ..., 0.       , 0.       ,
         0.       ], dtype=float32), 'jet1_eta': ndarray([-0.24857314, -0.5575249 ,  1.2823223 , ..., -1.755703  ,
         -0.41295132, -1.2447443 ], dtype=float32), 'jet1_phi': ndarray([-1.0920639 , -1.5882298 ,  1.3816643 , ...,  1.4520699 ,
          1.142174  , -0.96234006], dtype=float32), 'jet1_pt': ndarray([0.75420225, 1.0955306 , 1.1048746 , ..., 0.55651385, 0.8343586 ,
         0.73624724], dtype=float32), 'jet2_b-tag': ndarray([1.1074361, 2.2148721, 2.2148721, ..., 2.2148721, 0.       ,
         1.1074361], dtype=float32), 'jet2_eta': ndarray([-0.6536742 , -0.21364193,  1.540659  , ..., -0.12816104,
         -0.7818955 ,  0.1651938 ], dtype=float32), 'jet2_phi': ndarray([ 0.9303491,  1.2710146, -0.8196895, ..., -1.5614971, -1.1376071,
         -1.1969739], dtype=float32), 'jet2_pt': ndarray([1.3749921 , 0.8125812 , 0.8517372 , ...

In [13]:
import numpy as np
key_list=list(npy1.keys())
val_list=np.array(list(npy1.values()))

In [14]:
print(key_list)
print(val_list)
j=0
for i in key_list:
    j+=1
print(j)

['Type', 'jet1_b-tag', 'jet1_eta', 'jet1_phi', 'jet1_pt', 'jet2_b-tag', 'jet2_eta', 'jet2_phi', 'jet2_pt', 'jet3_b-tag', 'jet3_eta', 'jet3_phi', 'jet3_pt', 'jet4_b-tag', 'jet4_eta', 'jet4_phi', 'jet4_pt', 'lepton_eta', 'lepton_pT', 'lepton_phi', 'm_bb', 'm_jj', 'm_jjj', 'm_jlv', 'm_lv', 'm_wbb', 'm_wwbb', 'missing_energy_magnitude', 'missing_energy_phi']
[[ 1.          1.          1.         ...  1.          1.
   1.        ]
 [ 0.          2.1730762   0.         ...  0.          0.
   0.        ]
 [-0.24857314 -0.5575249   1.2823223  ... -1.755703   -0.41295132
  -1.2447443 ]
 ...
 [ 0.87667835  0.7983426   0.7801176  ...  0.91213906  0.80058527
   0.9691252 ]
 [ 0.32747006  1.4979699   0.45377317 ...  0.95725745  0.43221956
   0.7048917 ]
 [-0.6899932  -0.31300953  0.42562917 ...  0.02613812 -1.5219746
   0.88534594]]
29


In [15]:
columns=['m_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb', 'm_wbb', 'm_wwbb']
print(npy1[columns[0]])
Ncolumns=0
for i in columns:
    Ncolumns+=1
Nevents=29
print(Ncolumns)

[1.35376    0.3022199  0.90975326 ... 0.7873357  0.9221176  0.7312851 ]
7


In [16]:
rows, cols = (Nevents, Ncolumns)
# Creates a list containing Ncolumns lists, each of Nevents items

Matrix = [[0 for x in range(rows)] for y in range(cols)] 
for i in range(7):
    for j in range(cols):
        Matrix[i][j]=npy1[columns[j]]
print(Matrix)

[[ndarray([1.35376   , 0.3022199 , 0.90975326, ..., 0.7873357 , 0.9221176 ,
         0.7312851 ], dtype=float32), ndarray([0.9795631 , 0.83304816, 1.1083305 , ..., 0.97868377, 1.0380867 ,
         1.0617054 ], dtype=float32), ndarray([0.97807616, 0.98569965, 0.9856922 , ..., 0.98237854, 0.98750263,
         1.0720345 ], dtype=float32), ndarray([0.92000484, 0.9780984 , 0.95133126, ..., 1.1715132 , 0.9512442 ,
         0.78194284], dtype=float32), ndarray([0.72165745, 0.77973217, 0.8032515 , ..., 0.9685302 , 0.46553683,
         0.87044066], dtype=float32), ndarray([0.98875093, 0.99235576, 0.8659244 , ..., 0.9312682 , 0.80048835,
         1.1141888 ], dtype=float32), ndarray([0.87667835, 0.7983426 , 0.7801176 , ..., 0.91213906, 0.80058527,
         0.9691252 ], dtype=float32), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [ndarray([1.35376   , 0.3022199 , 0.90975326, ..., 0.7873357 , 0.9221176 ,
         0.7312851 ], dtype=float32), ndarray([0.9795631 , 0.83304816, 1

In [17]:
model.PrintRequiredInputTensors();

Model requires following inputs:
Fully Specified Tensor name: denseinput	type: float	shape: [1,7]


In [18]:
s = ROOT.TMVA_SOFIE_Higgs_trained_model.Session("Higgs_trained_model.dat")

In [21]:
i=0
for j in range(cols):
    mylis=Matrix[i][j]
    mylis = mylis[21:28].reshape(1,7)
    print(mylis)

[[0.75960004 0.86805    0.87948847 1.3357887  0.918797   0.8648047
  0.97312695]]
[[0.9803978  0.8289578  0.6159556  0.8690606  0.92730033 0.7482347
  0.65371543]]
[[0.9859478  1.0872251  1.7058299  0.98437315 0.98928845 1.1074071
  0.9867191 ]]
[[1.3758397  0.7822451  1.4349322  1.4588101  0.74353147 0.83487254
  0.6877329 ]]
[[1.3089557  0.87830997 0.59818643 0.9054574  0.8236121  0.6961423
  0.6523177 ]]
[[1.0808641  0.64220744 0.9112865  0.9495011  0.92282706 0.681356
  0.78359985]]
[[0.9565959  0.6209062  1.0071493  0.8634119  0.7984169  0.60438615
  0.7029413 ]]


In [19]:
i=0
for j in range(cols):
    mylis=Matrix[i][j]
    mylis = mylis[21:28].reshape(1,7)
    mylis = mylis.astype(np.float32)
    print("calling inference...")
    result = s.infer(mylis)
    print(result)
    

calling inference...
{ 0.570402f, 0.437383f }
calling inference...
{ 0.459884f, 0.549011f }
calling inference...
{ 0.825018f, 0.175621f }
calling inference...
{ 0.786556f, 0.211593f }
calling inference...
{ 0.423948f, 0.585387f }
calling inference...
{ 0.338189f, 0.663956f }
calling inference...
{ 0.280681f, 0.713514f }
